<a href="https://colab.research.google.com/github/ImronRosada/Data_Science/blob/main/customer_satisfaction_sentiment_analysis_ImronRosada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment: Customer Satisfaction and Sentiment Analysis

## Objective

* Generate NPS Category (Promoter, Passive, Detractor) based on the customer's likelihood to recommend
* Generate sentiment from customer's review, either using VADER or pre-trained deep learning model of your own choosing
* Save the processed data and create a dashboard to visualize the Response Rate, CSAT (either the overall score or the positive score), CES Score, NPS Score and customer's sentiment

**Change the Runtime to GPU for faster model computation**

In [ ]:
import pandas as pd
import numpy as np

pd.options.display.max_columns = 999
pd.options.display.float_format = "{:.2f}".format

## Data

Data survey of various [ticketing system](https://drive.google.com/file/d/1Qz9Knvaigjz0HGBFXtCkQwpyXPO8fi9_/view?usp=drive_link).

Description:

* id_survey: identifier
* date_of_survey: The date of the survey
* ticket_system: The name of the ticket system being reviewed
* overall_rating: The overall satisfaction rating given by the reviewer (out of 5)
* ease_of_use: How easy the ticket system is to use (out of 5)
* likelihood_to_recommend: The likelihood to recommend the system (out of 10)
* overall_text: The overall text of the review.



In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

data_path = '/content/drive/My Drive/Colab Notebooks/'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
file_name = data_path + 'ticket_system_review.csv'

df = pd.read_csv(file_name)

df.head()

,id_survey,date_of_survey,ticket_system,overall_rating,ease_of_use,likelihood_to_recommend,overall_text
0,T_04262,2024-12-19,Zoho Desk,NaN,NaN,NaN,NaN
1,T_04608,2024-12-25,ServiceNow,3.00,2.00,3.00,Pros:The pros are definetly the possibility to create personalized filters and manage tickets from various countries and clients.
2,T_03456,2024-12-04,Zendesk,NaN,NaN,NaN,NaN
3,T_02435,2024-11-13,Zoho Desk,4.00,3.00,10.00,Best help the executives instrument to give your client a brilliant encounterSubsequent to involving Zoho work area for over 2 years I feel this is one of the most amazing programming for our ticket and client experience the board. With the assistance of Zoho work area our group is profoundly useful and we are seeing an extraordinary change in our consumer loyalty rate and ticket goal time.
4,T_00495,2024-09-19,Freshdesk,4.00,5.00,10.00,"Pros:I love using Freshdesk. We evaluated 3 top-tier customer support management system products to manage our complex array of products and Freshdesk was the clear winner on functionality and price. While it's true, one of the contenders could have met our needs a tiny bit better. The price tag was 8x higher than Freshdesk.That's not a joke. From the moment I saw Freshdesk, I loved the simplistic design of the site. It reflects how they do all of their development. Features are added gracefully and in ways I don't always expect. For example, we had major problems with multiple agents working on the same ticket.Freshdesk added a very simple indicator that shows icons detailing that someone else is looking at that same ticket. It's very subtle, but very effective. All of their features are focused on high-usability, without cluttering the screen with additional controls. It reminds me of how Apple does UI. It's simple and elegant."


Change data type if necessary

In [ ]:
# Change data type if necessary
df.dtypes

,0
id_survey,object
date_of_survey,object
ticket_system,object
overall_rating,float64
ease_of_use,float64
likelihood_to_recommend,float64
overall_text,object


In [ ]:
# Convert data type
df['date_of_survey'] = pd.to_datetime(df['date_of_survey'])
#cek data type has been changed
df[['date_of_survey']].dtypes

,0
date_of_survey,datetime64[ns]


In [ ]:
df.sample(5)

,id_survey,date_of_survey,ticket_system,overall_rating,ease_of_use,likelihood_to_recommend,overall_text
1454,T_04607,2024-12-25,ServiceNow,4.00,5.00,9.00,"Best Service management tool which providesscalability, flexibility andgreat collaboration. It provides various feature CMDB(configuration management database), Knowledge database and intelligence workflow automation. It provides best ticket management."
307,T_01760,2024-10-28,Freshdesk,NaN,NaN,NaN,NaN
46,T_01923,2024-11-02,Zoho Desk,NaN,NaN,NaN,NaN
1490,T_00661,2024-09-25,otrs,NaN,NaN,NaN,NaN
568,T_03128,2024-11-28,Zoho Desk,NaN,NaN,NaN,NaN


## Fill Survey: Responded

Generate a column to indicate if a customer has filled/responded the survey (whether the overall_rating is blank or not).

In [ ]:
# How many customers responded to the survey?
df['fill_survey'] = np.where(df['overall_rating'].isnull(), 'Not Responded', 'Responded')
# Calculate the proportions
df.value_counts('fill_survey', normalize=True)

,proportion
fill_survey,
Responded,0.53
Not Responded,0.47


## Customer Satisfaction

Generate a column to indicate if a customer has satisfied with the service.

* Satisfied: overall rating 4-5
* Not Satisified: overall rating < 4


In [ ]:
# Adde 'customer_satisfaction' column based on 'overall_rating' value
df['customer_satisfaction'] = np.where(
    df['overall_rating'] >= 4, 'Satisfied',  # if rating >= 4, customer satisfied
    'Not Satisfied'  # if rating < 4, customer not satisfied
)
# Show the result
df[['overall_rating', 'customer_satisfaction']].sample(10) # sample 10 data

,overall_rating,customer_satisfaction
667,5.00,Satisfied
839,NaN,Not Satisfied
1634,NaN,Not Satisfied
1419,5.00,Satisfied
1309,5.00,Satisfied
1439,NaN,Not Satisfied
1258,NaN,Not Satisfied
49,NaN,Not Satisfied
62,NaN,Not Satisfied
340,5.00,Satisfied


Now we will measure the customer's overall satisfaction score (CSAT) with the following formula:

$$
Overall\ CSAT = \frac{\Sigma\ total\ satisfaction\ score}{number\ of\ responded\ customer \times \max\ rating}
$$

The max rating is inserted to convert the CSAT score into percentage. In the dataset, the max rating is 5 since the scale is from 1 to 5.

In [ ]:
# Create satisfaction category
# Responded Customer
responded_customer = df[ df['fill_survey'] == 'Responded'].copy()
max_rating = 5

csat_score_overall = responded_customer['overall_rating'].sum()/ (responded_customer.shape[0] * max_rating)

print(f'Overall CSAT Score: { (csat_score_overall * 100):.1f}')

Overall CSAT Score: 91.3


In [ ]:
responded_customer['overall_rating'].sum()

4084.0

In [ ]:
responded_customer.shape[0]

895

$$
Overall CSAT = \frac{4084.0}{895 \times 5} = \frac{4084.0}{4475} \approx 0.913
$$

Overall CSAT Score: `91.3` showed that the average satisfaction level of customers responding to the survey was approximately `91.3%` of the maximum possible score `(1-5 scale)`.

Calculate CSAT as the percentage of satisfied customer, with rating of 4-5 indicate that they are satisfied.

$$
Positive\ CSAT = \frac{number\ of\ satisfied\ customer}{number\ of\ responded\ customer}
$$

Both metrics are valid but they show different meaning. The overall CSAT shows the average satisfaction level of all customer while the Positive CSAT only focus on the percentage of satisfied customer.

If the survey has range of 1 - 5, customer can be classified as satisfied if the rating is 4 - 5.

In [ ]:
# How many customers are satisfied?
responded_customer['is_satisfied'] = np.where(responded_customer['overall_rating'] >= 4, 1, 0)
# Calculate the proportion of satisfied and dissatisfied customers
satisfaction_proportion = responded_customer.value_counts('is_satisfied', normalize=True)
#Show the result
satisfaction_proportion


,proportion
is_satisfied,
1,0.95
0,0.05


`95%` of customers who filled out the survey were satisfied with the service or product. `5%` of customers who filled out the survey were dissatisfied.

This shows that the majority of customers who responded to the survey had a high level of satisfaction, with only a small percentage being dissatisfied.

### CES Score

The formula for CES is similar to overall CSAT

$$
CES = \frac{\Sigma\ total\ effort\ score}{number\ of\ responded\ customer \times \max\ rating}
$$

Here we will calculate the CES score for ease of use (how easy the customer finds it to use the product or service) and likelihood to recommend (how likely the customer is to recommend the product or service to others

In [ ]:
# Check the maximum ease of use scale
responded_customer['ease_of_use'].describe()

,ease_of_use
count,895.00
mean,4.41
std,0.77
min,1.00
25%,4.00
50%,5.00
75%,5.00
max,5.00


In [ ]:
max_rating = 5  # Skala rating 1-5

# Calculating CES for Ease of Use
ces_ease_of_use = responded_customer['ease_of_use'].sum() / (responded_customer.shape[0] * max_rating)

# Show the result
print(f'Ease of Use CES Score: { (ces_ease_of_use * 100):.1f}')

Ease of Use CES Score: 88.2


In [ ]:
# Check the maximum likelihood_to_recommend
responded_customer['likelihood_to_recommend'].describe()

,likelihood_to_recommend
count,895.00
mean,8.55
std,1.56
min,1.00
25%,8.00
50%,9.00
75%,10.00
max,10.00


In [ ]:
max_rating = 10  # Skala rating 1-10

# Calculating CES for Likelihood to Recommend
ces_likelihood_to_recommend = responded_customer['likelihood_to_recommend'].sum() / (responded_customer.shape[0] * max_rating)

print(f'Likelihood to Recommend CES Score: { (ces_likelihood_to_recommend * 100):.1f}')


Likelihood to Recommend CES Score: 85.5


The CES Ease of Use score of `88.2` and the CES Likelihood to Recommend score of `85.5` indicate that most customers find it easy to use the product and tend to recommend it.

## NPS Score

Generate NPS Category based on the likelihood to recommend:

* Promoter: rating 9-10
* Passive: rating 7-8
* Detractor: rating < 7



In [ ]:
# Create NPS Category
nps_value = ['Promoter', 'Passive', 'Detractor']
nps_condition = [
    responded_customer['likelihood_to_recommend'] >= 9,  # Promoter: 9-10
    (responded_customer['likelihood_to_recommend'] >= 7) &
    (responded_customer['likelihood_to_recommend'] <= 8),  # Passive: 7-8
    responded_customer['likelihood_to_recommend'] < 7  # Detractor: <7
]
# Added 'nps_category' column
responded_customer['nps_category'] = np.select(nps_condition, nps_value)
# Calculate the proportion of each category
nps_proportion = responded_customer.value_counts(
    'nps_category', normalize=True)
# Displays results
nps_proportion

,proportion
nps_category,
Promoter,0.58
Passive,0.34
Detractor,0.08


- Promoter (58%): Most customers give a score of 9-10 for likelihood to recommend. This shows that the majority of customers are very satisfied and have the potential to recommend the product or service to others.
- Passive (34%): Some customers gave a score of 7-8. They tend to be satisfied but not completely enthusiastic about recommending a product or service.
- Detractor (8%): ​​Only a small percentage of customers gave a score of less than 7, indicating their dissatisfaction and the potential to leave a negative review.

Overall, with the majority of customers in the Promoter and Passive categories, it shows that the customer experience is generally positive, but the Detractor category needs to be considered to reduce the number of dissatisfied customers.

Calculate the NPS Score with the following formula

$$
NPS = \frac{Promoter - Detractor}{Total\ Survey\ Responded}
$$

In [ ]:
# NPS Score
nps_agg_1 = responded_customer.value_counts('nps_category').reset_index()
nps_agg_1.columns = ['nps_category', 'count']

# Take the number of Promoters and Detractors
nps_agg_promoter = nps_agg_1[nps_agg_1['nps_category'] == 'Promoter']['count'].item()
nps_agg_detractor = nps_agg_1[nps_agg_1['nps_category'] == 'Detractor']['count'].item()

# Calculate Total Respondents
total_respondents = nps_agg_1['count'].sum()

# Calculate NPS Score
nps_score = (nps_agg_promoter - nps_agg_detractor) / total_respondents

# Show the result NPS Score
print(f'NPS Score: { (nps_score * 100):.1f}%')

NPS Score: 50.2%


In [ ]:
nps_agg_1

,nps_category,count
0,Promoter,522
1,Passive,300
2,Detractor,73


`Total Survey Responded` = Promoter + Passive + Detractor = 522 + 300 + 73 = 895

$$
NPS = \frac{522 - 73}{895} = \approx 50.2%
$$

With an NPS score of `50.2%`, it can be concluded that:

Most customers are quite satisfied and enthusiastic about a product or service, so they are willing to recommend it to others (Promoters).
The proportion of dissatisfied customers (Detractors) is relatively small compared to Promoters, but it still needs to be considered to reduce the potential negative impact on reputation.
The NPS score of `50.2%` is considered good, but there is still room for improvement, especially by improving the customer experience in the Passive category so that they can become Promoters.
These results indicate that the product or service has a fairly positive image in the eyes of customers, but strategies to increase satisfaction need to be continuously implemented to achieve a higher score.

Join the additional columns to our main dataset.

In [ ]:
# Menggabungkan informasi dari responded_customer ke df_review
df = df.merge(responded_customer[['id_survey', 'is_satisfied', 'nps_category']], on='id_survey', how='left')
df.sample(5)

,id_survey,date_of_survey,ticket_system,overall_rating,ease_of_use,likelihood_to_recommend,overall_text,fill_survey,customer_satisfaction,is_satisfied,nps_category
1408,T_03312,2024-12-01,Jira Service Management,NaN,NaN,NaN,NaN,Not Responded,Not Satisfied,NaN,NaN
953,T_00595,2024-09-23,Freshdesk,5.00,4.00,8.00,"all problems ar mentioned in the cons, overall we are satisfied",Responded,Satisfied,1.00,Passive
508,T_03798,2024-12-10,Freshdesk,5.00,5.00,10.00,"Fantastic. Customer support is great, and that's why we stuck with them for so long. The pricing is competitive, and in most cases, cheaper than Zendesk with all of the Zendesk functionality.",Responded,Satisfied,1.00,Promoter
829,T_02229,2024-11-09,Zoho Desk,5.00,5.00,10.00,My experience everytime is a good one. I love it all over again when I use Zoho Desk every day of the week,Responded,Satisfied,1.00,Promoter
548,T_04198,2024-12-17,Zendesk,3.00,3.00,5.00,It looks good but perhaps not in my personal taste to use.,Responded,Not Satisfied,0.00,Detractor


## Sentiment



We also want to check the sentiment of comments given by customer during the survey.

First we filter out any blank comment from the data.

In [ ]:
# Display long text without being cut
pd.set_option('display.max_colwidth', None)
# Filter data to only keep rows with existing comments
df_clean = df.dropna(subset=['overall_text'])[['id_survey', 'overall_text']]
# Show the result
df_clean.head()

,id_survey,overall_text
1,T_04608,Pros:The pros are definetly the possibility to create personalized filters and manage tickets from various countries and clients.
3,T_02435,Best help the executives instrument to give your client a brilliant encounterSubsequent to involving Zoho work area for over 2 years I feel this is one of the most amazing programming for our ticket and client experience the board. With the assistance of Zoho work area our group is profoundly useful and we are seeing an extraordinary change in our consumer loyalty rate and ticket goal time.
4,T_00495,"Pros:I love using Freshdesk. We evaluated 3 top-tier customer support management system products to manage our complex array of products and Freshdesk was the clear winner on functionality and price. While it's true, one of the contenders could have met our needs a tiny bit better. The price tag was 8x higher than Freshdesk.That's not a joke. From the moment I saw Freshdesk, I loved the simplistic design of the site. It reflects how they do all of their development. Features are added gracefully and in ways I don't always expect. For example, we had major problems with multiple agents working on the same ticket.Freshdesk added a very simple indicator that shows icons detailing that someone else is looking at that same ticket. It's very subtle, but very effective. All of their features are focused on high-usability, without cluttering the screen with additional controls. It reminds me of how Apple does UI. It's simple and elegant."
5,T_02190,"Freshdesk has been a great product to use for us, it's easy to use and set up and provides a much needed cost effective service."
6,T_00278,"Zoho Desk delivers attributes that are quality, and well captured to offer customer help desk procedures"


### Text Cleansing

In order to get more accurate sentiment, several text cleansing need to be done. In most of recent sentiment analysis models and algorithm, the only text cleansing needed are as follows:

* Clean double whitespace
* Clean URL/website
* Clean username (mostly in social media or digital text)

In [ ]:
# Define/create a text cleansing function
import re

def cleansing_text(x):
  # clean double whitespace
  out_text = ' '.join(x.split())

  # clean url
  out_text = re.sub(r"http\S+|www\S+|https\S+", 'http', out_text)

  # clean username
  out_text = re.sub(r"@\S+", '@user', out_text)

  return(out_text)

cleansing_text(" Doesn't  dissapoint. The car       was great. It was the best car rental experiences I've had! Salute to @jone who recommend https:/rental.com")

"Doesn't dissapoint. The car was great. It was the best car rental experiences I've had! Salute to @user who recommend http"

In [ ]:
# Display long text without being cut
pd.set_option('display.max_colwidth', None)
# Apply cleansing function to column 'overall_text' and create new column 'clean_text'
df_clean['clean_text'] = df_clean['overall_text'].apply(cleansing_text)
# Show the result
df_clean.tail()


,id_survey,overall_text,clean_text
1675,T_02606,"Pros:Jira Service Desk is so easy to use and highly customizable. You can manage workflows, components and many other things. It is easy to track queries and keep the team informed about the progress of all the work.","Pros:Jira Service Desk is so easy to use and highly customizable. You can manage workflows, components and many other things. It is easy to track queries and keep the team informed about the progress of all the work."
1676,T_04746,Helps us keep track of issues and use metrics for our team.,Helps us keep track of issues and use metrics for our team.
1677,T_01816,It's good overall. They just need to work on getting out bugs better,It's good overall. They just need to work on getting out bugs better
1678,T_02690,Pros:Great centralized management of tickets/requests. Fun features built in for both managers and technicians (scoreboard and point system).Built-in surveys make it easy to gather feedback from clients and users.,Pros:Great centralized management of tickets/requests. Fun features built in for both managers and technicians (scoreboard and point system).Built-in surveys make it easy to gather feedback from clients and users.
1680,T_01821,"Its a great software becuase it works! I have used software with all kinds of bells and whistles that barely get the job done. Matter a fact, here comes a ticket now!","Its a great software becuase it works! I have used software with all kinds of bells and whistles that barely get the job done. Matter a fact, here comes a ticket now!"


### Sentiment Analysis

Create a sentiment categories using algorithm of your own choice.

### Lexicon-based Model with VADER

The most simple sentiment analysis consists of algorithms focusing on words that match with their lexicon database. These algorithm only look at words or tokens that appear on each text, with each words may have different sentiment score ("worst" may have stronger negative sentiment compared to "bad"). However, this method may not be able to fully capture the context of a sentence or paragraphs.

Currently the popular algorithm is is VADER (Valence Aware Dictionary and sEntiment Reasoner).

Documentation: https://github.com/cjhutto/vaderSentiment

In [ ]:
# Download or prepare required library for sentiment analysis
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

# Download VADER lexicon for sentiment score
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

Create a sentiment analyzer and collect sentiment score for each comment.

VADER will return sentiment score ranging from -1 (negative) to 1 (positive). The author suggest to categorize the sentiment with the following conditions:

* Negative: score <= - 0.05
* Neutral: score -0.05 to 0.05
* Positive: score >= 0.05



In [ ]:
# Initialize the Sentiment Intensity Analyzer
model_sia = SentimentIntensityAnalyzer()

# Get polarity score
df_clean['vader_score'] = df_clean['clean_text'].apply(model_sia.polarity_scores)
df_clean['vader_compound'] = df_clean['vader_score'].apply(lambda x: x['compound'])

# Create sentiment category
sent_value = ['Positive', 'Neutral', 'Negative']
sent_condition = [df_clean['vader_compound'] > 0.05, # Positive: score >= 0.05
                  (df_clean['vader_compound'] <= 0.05) & (df_clean['vader_compound'] >= - 0.05), # Neutral: -0.05 <= score <= 0.05
                  df_clean['vader_compound'] < -0.05 # Negative: score <= -0.05
                  ]
df_clean['vader_sentiment'] = np.select(sent_condition, sent_value)
# Show the Result
df_clean.sample(5)

,id_survey,overall_text,clean_text,vader_score,vader_compound,vader_sentiment
35,T_02657,Efficient tool to manage tickets.,Efficient tool to manage tickets.,"{'neg': 0.0, 'neu': 0.588, 'pos': 0.412, 'compound': 0.4215}",0.42,Positive
454,T_03950,Its has been a one stop flow of service and easy to use once it hands on.,Its has been a one stop flow of service and easy to use once it hands on.,"{'neg': 0.115, 'neu': 0.733, 'pos': 0.152, 'compound': 0.1779}",0.18,Positive
1250,T_03819,Pros:I like how organized and customizable this software is. My company was able to place custom macros for responding emails. This allowed for responses to be sent out conveniently faster.,Pros:I like how organized and customizable this software is. My company was able to place custom macros for responding emails. This allowed for responses to be sent out conveniently faster.,"{'neg': 0.0, 'neu': 0.921, 'pos': 0.079, 'compound': 0.3612}",0.36,Positive
882,T_04731,"Pros:I love each andevery serviceprovided by zoho. Be it CRM, Desk, invoices etc. In terms of support these guys are good. Right now i am using CRM and invoices and loved the UI/UX.","Pros:I love each andevery serviceprovided by zoho. Be it CRM, Desk, invoices etc. In terms of support these guys are good. Right now i am using CRM and invoices and loved the UI/UX.","{'neg': 0.0, 'neu': 0.671, 'pos': 0.329, 'compound': 0.9287}",0.93,Positive
457,T_03077,"We needed a way to take in user issues and then sort them into the teams that need to handle them. Service desk provides a great interface for our current issue tracking system, so it was a no-brainer.","We needed a way to take in user issues and then sort them into the teams that need to handle them. Service desk provides a great interface for our current issue tracking system, so it was a no-brainer.","{'neg': 0.0, 'neu': 0.892, 'pos': 0.108, 'compound': 0.6249}",0.62,Positive


Check the number of data by the sentiment.

In [ ]:
df_clean.value_counts('vader_sentiment')

,count
vader_sentiment,
Positive,782
Neutral,86
Negative,27


In [ ]:
# Filter data vader_sentiment 'Positive'
df_clean[df_clean['vader_sentiment'] == 'Positive'].sample(5)

,id_survey,overall_text,clean_text,vader_score,vader_compound,vader_sentiment
892,T_02630,"Easy to pick up and use right away; minimal training required. I appreciate having a centralized location to view, manage, and reply to all of my tickets. It's a benefit if you can combine prepared replies with your own expertise.","Easy to pick up and use right away; minimal training required. I appreciate having a centralized location to view, manage, and reply to all of my tickets. It's a benefit if you can combine prepared replies with your own expertise.","{'neg': 0.0, 'neu': 0.759, 'pos': 0.241, 'compound': 0.8591}",0.86,Positive
838,T_02135,"Pros:A very good customer support application with a lot of features, the template especially makes ticketing very easy and fast. The software allows for integration with other application. Awesome product!","Pros:A very good customer support application with a lot of features, the template especially makes ticketing very easy and fast. The software allows for integration with other application. Awesome product!","{'neg': 0.0, 'neu': 0.645, 'pos': 0.355, 'compound': 0.9293}",0.93,Positive
1,T_04608,Pros:The pros are definetly the possibility to create personalized filters and manage tickets from various countries and clients.,Pros:The pros are definetly the possibility to create personalized filters and manage tickets from various countries and clients.,"{'neg': 0.0, 'neu': 0.89, 'pos': 0.11, 'compound': 0.2732}",0.27,Positive
685,T_01045,Very good. Using for all our support tickets and interactions.,Very good. Using for all our support tickets and interactions.,"{'neg': 0.0, 'neu': 0.576, 'pos': 0.424, 'compound': 0.7089}",0.71,Positive
140,T_01026,"Pros:- Intuitive and easy to use. - Well designed UI. Pleasant to the eye.I always knew what to do next.The way that it is designed makes it easy for me to know what is my next step always.- It has apps that can be integrated. For my case, I was able to integrate with chat software Zopim.- It has an analytics feature that we used to keep track of our support teams performance. - Low start-up cost with high functionality. - Integrated with our phone service.","Pros:- Intuitive and easy to use. - Well designed UI. Pleasant to the eye.I always knew what to do next.The way that it is designed makes it easy for me to know what is my next step always.- It has apps that can be integrated. For my case, I was able to integrate with chat software Zopim.- It has an analytics feature that we used to keep track of our support teams performance. - Low start-up cost with high functionality. - Integrated with our phone service.","{'neg': 0.023, 'neu': 0.826, 'pos': 0.151, 'compound': 0.8957}",0.90,Positive


In [ ]:
# Filter data vader_sentiment 'Negative'
df_clean[df_clean['vader_sentiment'] == 'Negative'].sample(5)

,id_survey,overall_text,clean_text,vader_score,vader_compound,vader_sentiment
1201,T_03649,Not as easy as it should be to find what I am looking for on the site.,Not as easy as it should be to find what I am looking for on the site.,"{'neg': 0.138, 'neu': 0.862, 'pos': 0.0, 'compound': -0.3412}",-0.34,Negative
1611,T_03923,"Overall, Freshdesk is intuitive to use and I have had no problem providing quality customer support to our clients and customers with this platform.","Overall, Freshdesk is intuitive to use and I have had no problem providing quality customer support to our clients and customers with this platform.","{'neg': 0.178, 'neu': 0.725, 'pos': 0.098, 'compound': -0.296}",-0.30,Negative
901,T_03293,"My Service Provider Specialist team utilizes Zendesk daily to track tickets submitted by internal and external users. I am able to manage and report on open ticket times, see real time ticket queue counts, and assign tickets as needed to to members. Overall it has been a very smooth process but when Zendesk goes down it causes a lot of confusion an frustration to outside parties attempting to communicate to our company","My Service Provider Specialist team utilizes Zendesk daily to track tickets submitted by internal and external users. I am able to manage and report on open ticket times, see real time ticket queue counts, and assign tickets as needed to to members. Overall it has been a very smooth process but when Zendesk goes down it causes a lot of confusion an frustration to outside parties attempting to communicate to our company","{'neg': 0.091, 'neu': 0.863, 'pos': 0.046, 'compound': -0.5267}",-0.53,Negative
847,T_02003,"Pros:At the first opening, it's a little scary. There are many menus in all directions, but after a few moments of use, the logic of the application is simple to handle.","Pros:At the first opening, it's a little scary. There are many menus in all directions, but after a few moments of use, the logic of the application is simple to handle.","{'neg': 0.065, 'neu': 0.935, 'pos': 0.0, 'compound': -0.2391}",-0.24,Negative
433,T_01249,"Pros:You could add as much details to a ticket, however, it is not a good platform for internal communication between teams","Pros:You could add as much details to a ticket, however, it is not a good platform for internal communication between teams","{'neg': 0.118, 'neu': 0.882, 'pos': 0.0, 'compound': -0.3412}",-0.34,Negative


In [ ]:
# Filter data vader_sentiment 'Neutral'
df_clean[df_clean['vader_sentiment'] == 'Neutral'].sample(5)

,id_survey,overall_text,clean_text,vader_score,vader_compound,vader_sentiment
566,T_01367,Pros:Ticketing systemAnalyticsZoho ticketautomation,Pros:Ticketing systemAnalyticsZoho ticketautomation,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.00,Neutral
852,T_01835,"Pros:Ease of use, customer service, brand modifications","Pros:Ease of use, customer service, brand modifications","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.00,Neutral
734,T_02712,"Document and analyze response times to incidents and projects, and with this data measure the quality of services provided to users.","Document and analyze response times to incidents and projects, and with this data measure the quality of services provided to users.","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.00,Neutral
275,T_00201,"Overall, can't imagine using a different software in replacement of Zoho Desk. It gives everything I need for work.","Overall, can't imagine using a different software in replacement of Zoho Desk. It gives everything I need for work.","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.00,Neutral
294,T_00825,Pros:Jira service desk is used to log and track the issues we are facing during the training preparation.,Pros:Jira service desk is used to log and track the issues we are facing during the training preparation.,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.00,Neutral


### Deep Learning with ROBERTA

A more advanced approach to sentiment analysis is using a pre-trained model, a machine learning model that has been trained or learning from another dataset, typically a large dataset, and apply the model to predict our own dataset.

There are many models that can do sentiment analysis, but here we will use one as an example

Documentation: https://huggingface.co/cardiffnlp/twitter-roberta-base-sentiment

The output can be differ among pre-trained models:

* *distilbert-base-uncased-finetuned-sst-2-english*: positive/negative
* *twitter-roberta-base-sentiment*: positive/neutral/negative
* *bert-base-uncased-emotion*: sadness/joy/anger/love/fear/surprise

In [ ]:
%%capture
!pip install transformers
from transformers import pipeline
# sentiment analysis task with twitter roberta model
sentiment_pipeline = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")

In [ ]:
%%time
# Predict sentiment
transformer_results = sentiment_pipeline(df_clean['clean_text'].tolist())

df_clean['transformer_score'] = transformer_results
df_clean['transformer_sentiment'] = df_clean['transformer_score'].apply(lambda x: x['label'])

# Create sentiment category
sent_value = ['Negative', 'Neutral', 'Positive']
sent_condition = [df_clean['transformer_sentiment'] == 'LABEL_0',
                  df_clean['transformer_sentiment'] == 'LABEL_1',
                  df_clean['transformer_sentiment'] == 'LABEL_2'
                  ]

df_clean['transformer_sentiment'] = np.select(sent_condition, sent_value)

df_clean.sample(5)

CPU times: user 2min 55s, sys: 333 ms, total: 2min 56s
Wall time: 3min 22s


,id_survey,overall_text,clean_text,vader_score,vader_compound,vader_sentiment,transformer_score,transformer_sentiment
679,T_01043,Pros:easy to use and program. Love this CRM. It is one of the best I have ever used.It is easy to configure to meet your needs and the service is absolutely awesome.I have been using this program for over 7 years.,Pros:easy to use and program. Love this CRM. It is one of the best I have ever used.It is easy to configure to meet your needs and the service is absolutely awesome.I have been using this program for over 7 years.,"{'neg': 0.0, 'neu': 0.761, 'pos': 0.239, 'compound': 0.9062}",0.91,Positive,"{'label': 'LABEL_2', 'score': 0.9913234710693359}",Positive
845,T_02002,"The ability to communicate with variouscategories of people- customers, suppliers and more","The ability to communicate with variouscategories of people- customers, suppliers and more","{'neg': 0.0, 'neu': 0.827, 'pos': 0.173, 'compound': 0.3182}",0.32,Positive,"{'label': 'LABEL_1', 'score': 0.7978549003601074}",Neutral
1124,T_00352,"Freshdeskfrom our users is crucial as a consumer app. With Freshdesk, our customers can quickly and easily send us comments or concerns, and my staff can quickly and simply respond and stay in touch with the customer. Each feedback ticket may then be monitored to ensure it was addressed and resolved in a timely manner.","Freshdeskfrom our users is crucial as a consumer app. With Freshdesk, our customers can quickly and easily send us comments or concerns, and my staff can quickly and simply respond and stay in touch with the customer. Each feedback ticket may then be monitored to ensure it was addressed and resolved in a timely manner.","{'neg': 0.0, 'neu': 0.882, 'pos': 0.118, 'compound': 0.6908}",0.69,Positive,"{'label': 'LABEL_2', 'score': 0.6339832544326782}",Positive
850,T_03791,"My experience with Freshdesk (4 years) is only positive.If I have a concern, it's addressed by fast and with grace by Freshdesk's Product Support team.Freshdesk's commitment to excellence makes our team's job simple in providing excellent customer support to our clients and their customers.","My experience with Freshdesk (4 years) is only positive.If I have a concern, it's addressed by fast and with grace by Freshdesk's Product Support team.Freshdesk's commitment to excellence makes our team's job simple in providing excellent customer support to our clients and their customers.","{'neg': 0.0, 'neu': 0.659, 'pos': 0.341, 'compound': 0.9559}",0.96,Positive,"{'label': 'LABEL_2', 'score': 0.9435489773750305}",Positive
632,T_04509,"It is a great product, but still lots of improvements needs to be made","It is a great product, but still lots of improvements needs to be made","{'neg': 0.0, 'neu': 0.667, 'pos': 0.333, 'compound': 0.6705}",0.67,Positive,"{'label': 'LABEL_2', 'score': 0.8828734755516052}",Positive


In [ ]:
df_clean.value_counts('transformer_sentiment')

,count
transformer_sentiment,
Positive,785
Neutral,92
Negative,18


In [ ]:
df_clean[df_clean['vader_sentiment'] != df_clean['transformer_sentiment']].tail()

,id_survey,overall_text,clean_text,vader_score,vader_compound,vader_sentiment,transformer_score,transformer_sentiment
1630,T_00263,One stop shop,One stop shop,"{'neg': 0.524, 'neu': 0.476, 'pos': 0.0, 'compound': -0.296}",-0.30,Negative,"{'label': 'LABEL_1', 'score': 0.6285418272018433}",Neutral
1657,T_03627,"It provided us with a great tool for supporting our users, but the support from Zendesk became lacking and so we moved on to a company that had better customer support.","It provided us with a great tool for supporting our users, but the support from Zendesk became lacking and so we moved on to a company that had better customer support.","{'neg': 0.0, 'neu': 0.608, 'pos': 0.392, 'compound': 0.9377}",0.94,Positive,"{'label': 'LABEL_1', 'score': 0.409112811088562}",Neutral
1665,T_01497,Integration and flexibility,Integration and flexibility,"{'neg': 0.0, 'neu': 0.455, 'pos': 0.545, 'compound': 0.34}",0.34,Positive,"{'label': 'LABEL_1', 'score': 0.6647835373878479}",Neutral
1666,T_02696,It just works.,It just works.,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}",0.00,Neutral,"{'label': 'LABEL_2', 'score': 0.7977412343025208}",Positive
1676,T_04746,Helps us keep track of issues and use metrics for our team.,Helps us keep track of issues and use metrics for our team.,"{'neg': 0.0, 'neu': 0.809, 'pos': 0.191, 'compound': 0.3818}",0.38,Positive,"{'label': 'LABEL_1', 'score': 0.6566141247749329}",Neutral


## Save Results

Save the review data with NPS category and sentiment information to new csv.

In [ ]:
# Menggabungkan informasi dari df_clean ke df yang di simpan pada df_final
df_final = df.merge(df_clean[['id_survey', 'vader_sentiment', 'transformer_sentiment']], on = 'id_survey', how = 'left')
df_final.sample(5)

,id_survey,date_of_survey,ticket_system,overall_rating,ease_of_use,likelihood_to_recommend,overall_text,fill_survey,customer_satisfaction,is_satisfied,nps_category,vader_sentiment,transformer_sentiment
205,T_02513,2024-11-14,Zendesk,4.00,5.00,8.00,Pros:Very easy to understand and a great tool for quick and simple customer resolutions and troubleshooting.,Responded,Satisfied,1.00,Passive,Positive,Positive
1043,T_03623,2024-12-07,Zendesk,NaN,NaN,NaN,NaN,Not Responded,Not Satisfied,NaN,NaN,NaN,NaN
1186,T_01728,2024-10-27,Freshdesk,5.00,5.00,10.00,Overall experience has been great. We were able to integrate multiple products onto one platform. The integration with various support channels is great.,Responded,Satisfied,1.00,Promoter,Positive,Positive
1050,T_01064,2024-10-08,Freshdesk,5.00,5.00,10.00,I recommend this app because it is really useful especially in communicating!!,Responded,Satisfied,1.00,Promoter,Positive,Positive
1451,T_03389,2024-12-03,Zoho Desk,5.00,5.00,8.00,"Pros:Getting control of the team email communications. Allowing for notes, comments, and accountability for the entire team.",Responded,Satisfied,1.00,Passive,Neutral,Positive


In [ ]:
# Load data to excel
# Save the DataFrame to an Excel file
df_final.to_excel('ticket_system_review_update.xlsx', index=False)
print("DataFrames successfully saved to excel !!")

DataFrames successfully saved to excel !!
